In [2]:
import pandas as pd
import os

In [3]:
folder_path = "/Users/divyanshdoshi/Desktop/DCU/dmv/Data"

In [4]:
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
print(all_files)

['flightlist_20200801_20200831.csv', 'flightlist_20210901_20210930.csv', 'flightlist_20191101_20191130.csv', 'flightlist_20190201_20190228.csv', 'flightlist_20191201_20191231.csv', 'flightlist_20190901_20190930.csv', 'flightlist_20211101_20211130.csv', 'flightlist_20210201_20210228.csv', 'flightlist_20211201_20211231.csv', 'flightlist_20220501_20220531.csv', 'flightlist_20220301_20220331.csv', 'flightlist_20201001_20201031.csv', 'flightlist_20220601_20220630.csv', 'flightlist_20190801_20190831.csv', 'flightlist_20211001_20211031.csv', 'flightlist_20201201_20201231.csv', 'flightlist_20220101_20220131.csv', 'flightlist_20220401_20220430.csv', 'flightlist_20201101_20201130.csv', 'flightlist_20200201_20200229.csv', 'flightlist_20220701_20220731.csv', 'flightlist_20200901_20200930.csv', 'flightlist_20210801_20210831.csv', 'flightlist_20191001_20191031.csv', 'flightlist_20190501_20190531.csv', 'flightlist_20190601_20190630.csv', 'flightlist_20190301_20190331.csv', 'flightlist_20220901_202209

In [5]:
import pandas as pd
import os

folder_path = "/Users/divyanshdoshi/Desktop/DCU/dmv/Data"
all_files = os.listdir(folder_path)

chunks = []

for file in all_files:
    file_path = os.path.join(folder_path, file)

    # Skip directories and non-csv files
    if not os.path.isfile(file_path) or not file.lower().endswith(".csv"):
        continue

    for chunk in pd.read_csv(
        file_path,
        chunksize=100_000,
        low_memory=False,
        encoding="latin1",
        dtype=str
    ):
        chunks.append(chunk)

flights_all = pd.concat(chunks, ignore_index=True)


In [6]:
print(flights_all.info())
print(flights_all.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117258215 entries, 0 to 117258214
Data columns (total 16 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   callsign      object
 1   number        object
 2   icao24        object
 3   registration  object
 4   typecode      object
 5   origin        object
 6   destination   object
 7   firstseen     object
 8   lastseen      object
 9   day           object
 10  latitude_1    object
 11  longitude_1   object
 12  altitude_1    object
 13  latitude_2    object
 14  longitude_2   object
 15  altitude_2    object
dtypes: object(16)
memory usage: 14.0+ GB
None
  callsign number  icao24 registration typecode origin destination  \
0  ATN0310    NaN  a44a22       N376AN     B763   RJTJ        13CL   
1   CSN311  CZ311  78149f       B-1169     B789    NaN        CYYZ   
2   SXY335    NaN  4cc3c9       TF-AMI     B744   FAOR        EGCN   
3   AVA150    NaN  aabbd0       N791AV     B788   SKMA         NaN   
4  ANZ1125    NaN 

In [7]:
flights = flights_all[['origin', 'day']].copy()


In [8]:
flights = flights.dropna(subset=['origin', 'day'])


In [9]:
flights['day'] = pd.to_datetime(flights['day'])


In [10]:
airports = pd.read_csv("/Users/divyanshdoshi/Desktop/DCU/dmv/world-airports.csv")[['ident', 'iso_country']]
airports.rename(columns={'ident': 'origin', 'iso_country': 'country'}, inplace=True)

# merge with flights
flights = flights.merge(airports, on='origin', how='left')

# remove rows with unknown countries
flights = flights.dropna(subset=['country'])


In [11]:
daily_flights = flights.groupby(['day', 'country']).size().reset_index(name='flights')


In [12]:
!pip install pycountry_convert


In [13]:
import pycountry_convert as pc

def country_to_continent(country_code):
    try:
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        return continent_code
    except:
        return None

daily_flights['continent'] = daily_flights['country'].apply(country_to_continent)


In [14]:
daily_flights.to_csv("opensky_flights_cleaned.csv", index=False)


In [15]:
from IPython.display import FileLink, display

# Display a download link
FileLink('opensky_flights_cleaned.csv')

/Users/divyanshdoshi/Desktop/DCU/dmv/opensky_flights_cleaned.csv